# Exercise

## super cell expansion

- load Au.cif cif file and convert it to a 2x2x2 supercell

In [ ]:
# hints:
# see diffpy.structure.loadStructure function and
# diffpy.structure.expansion module.

## particle generation from crystal structure

- create a spherical cutout (radius = 8A) from Au.cif crystal structure

In [ ]:
# hints: see diffpy.structure.expansion module
from diffpy.structure import loadStructure
from diffpy.structure.expansion.makeellipsoid import makeSphere

aubulk = loadStructure('Au.cif')
goldsphere = makeSphere(aubulk, 8)
goldsphere.write('goldsphere.xyz', 'xyz')
print(len(goldsphere))

## make cylindrical layer

- load Au.cif cif file and expand it to a cylindrical layer with R=30A, thickness=5A.
- no direct function to do this - we need to create a big enough block and remove atoms outside of the desired cylinder volume

In [ ]:
# three step process
# (1) create a supercell block of sufficient size
# (2) find atom nearest to the center of mass
# (3) remove any atoms outside of the desired cylinder
import numpy as np
a = aubulk.lattice.a
radius = 30
thickness = 4
MXY = 2 * int(np.ceil(radius / a))
MZ = int(np.ceil(thickness / a + 1))
print("block multipliers along xy and z", MXY, MZ)

block = supercell(aubulk, [MXY, MXY, MZ])
center_cartn = block.xyz_cartn.mean(axis=0)
print("center:", center_cartn)
vfromcenter = block.xyz_cartn - center_cartn
zoffset = vfromcenter[:, 2]
roffset = np.power(vfromcenter[:,:2], 2).sum(axis=1)**0.5
isoutside = (np.abs(zoffset) > thickness / 2)
isoutside = np.logical_or(isoutside, (roffset > radius))
print("full block atom count:", len(block))
cylinder = block - block[isoutside]
print("cylinder atom count:", len(cylinder))
cylinder.write('cylinder.xyz', 'xyz')

## extract molecule from a CIF file

- extract single quinacridone molecule from the quinacridone-beta.cif structure

In [ ]:
%matplotlib notebook
import matplotlib
matplotlib.rc('figure', figsize=(5, 3.75))
from matplotlib.pyplot import subplots
from diffpy.srreal.overlapcalculator import OverlapCalculator
from diffpy.srreal.pdfcalculator import PDFCalculator
from diffpy.srreal.structureadapter import nosymmetry
from diffpy.structure import loadStructure
from diffpy.structure.expansion import supercell

pc = PDFCalculator()
pc.baseline = 'zero'
qb = loadStructure('quinacridone-beta.cif')
qb222 = supercell(qb, [2,2,2])

qb.Uisoequiv = 0.003
oc = OverlapCalculator()
oc.atomradiitable.setDefault(0.75)
oc(nosymmetry(qb222))
print([len(n) for n in oc.neighborhoods])
